In [2]:
from google.cloud import bigquery
import src.config as config

In [4]:
PROJECT_ID = config.BQ_PROJECT
DATASET_ID = config.BQ_DATASET
TABLE_ID = "c02_delta"
PARTITION_MONTH = "202108" # El mes que quieres analizar

# try:
#     sql_query = generate_average_delta_query(PROJECT_ID, DATASET_ID, TABLE_ID, PARTITION_MONTH)
#     print("--- Consulta SQL Generada ---")
#     print(sql_query)
#     print("-----------------------------")
#
#     # Opcional: Ejecutar la consulta directamente en BigQuery (descomentar para usar)
#     client = bigquery.Client(project=config.BQ_PROJECT)
#     # print("Ejecutando consulta en BigQuery...")
#     query_job = client.query(sql_query)
#     results = query_job.result()

In [17]:
def build_momentum_alter_update_script(
    table_id: str,
    feature_names: list[str],
    max_delta: int = 3,
    column_type: str = "FLOAT64",
) -> str:
    """
    Genera un script de BigQuery que:
      1) Agrega columnas *_momentum_ponderado a la tabla.
      2) Actualiza esas columnas con el cálculo de momentum ponderado
         en base a los deltas *_delta_1..*_delta_max_delta.

    Args:
        table_id: ID completo de la tabla en BigQuery, ej: "proyecto.dataset.c02_delta".
        feature_names: lista de features base, ej: ["edad", "altura"].
        max_delta: cantidad de deltas disponibles (ej. 3 → _delta_1.._delta_3).
        column_type: tipo de dato de la nueva columna (por defecto FLOAT64).

    Returns:
        str: script SQL listo para ejecutar en BigQuery.
    """

    alter_statements = []
    update_assignments = []

    for feat in feature_names:
        # nombre de la nueva columna
        alias = f"{feat}_momentum_ponderado"

        # construyo la expresión de momentum ponderado
        terms = []
        # pesos: max_delta para delta_1, ..., 1 para delta_max_delta
        for k in range(1, max_delta + 1):
            weight = max_delta - k + 1
            col_name = f"{feat}_delta_{k}"
            terms.append(f"{weight} * {col_name}")

        expr = " + ".join(terms)

        # ALTER TABLE para agregar la columna (si no existe)
        alter_statements.append(
            f"ALTER TABLE `{table_id}` "
            f"ADD COLUMN IF NOT EXISTS {alias} {column_type};"
        )

        # assignment para el UPDATE
        update_assignments.append(f"  {alias} = {expr}")

    # Bloque de ALTERs (uno por columna)
    alter_block = "\n".join(alter_statements)

    # Bloque de UPDATE (un único UPDATE seteando todas)
    update_block = ",\n".join(update_assignments)

    script = f"""
{alter_block}

UPDATE `{table_id}`
SET
{update_block} where 1 = 1;
"""
    return script.strip()



In [18]:
query = build_momentum_alter_update_script(table_id=f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}", feature_names=['Master_Finiciomora',
'Master_Fvencimiento',
'Master_fultimo_cierre',
'Visa_Finiciomora',
'Visa_fultimo_cierre',
'Visa_mpagado',
'cpayroll_trx',
'mcaja_ahorro_adicional',
'mcomisiones',
'mcomisiones_mantenimiento',
'mcomisiones_otras',
'mpayroll',
'mrentabilidad',
'mtransferencias_recibidas']
)

In [19]:
client = bigquery.Client(project=config.BQ_PROJECT)
# print("Ejecutando consulta en BigQuery...")
query_job = client.query(query)
results = query_job.result()


BadRequest: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/dmecoyfin-250928192534125/queries/8d30ac6e-b6a1-45a6-b5a1-aa8504d64f39?maxResults=0&location=US&prettyPrint=false: Exceeded rate limits: too many table update operations for this table. For more information, see https://cloud.google.com/bigquery/troubleshooting-errors at [14:1]

Location: US
Job ID: 8d30ac6e-b6a1-45a6-b5a1-aa8504d64f39
